In [33]:
import nba_api
import numpy as np
import pandas as pd
import requests
import time

In [22]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, franchiseplayers, playerestimatedmetrics, commonteamroster, commonallplayers

In [30]:
player_df = pd.DataFrame(players.get_players())
player_df.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [18]:
teams_df = pd.DataFrame(teams.get_teams())
teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [36]:
franchise_players = franchiseplayers.FranchisePlayers(team_id=1610612737)

In [13]:
#What we care about is PEM
pem = playerestimatedmetrics.PlayerEstimatedMetrics(season='1996-97')

In [14]:
pem.get_data_frames()[0]

,PLAYER_ID,PLAYER_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_AST_RATIO,E_OREB_PCT,E_DREB_PCT,E_REB_PCT,E_TOV_PCT,E_USG_PCT,E_PACE,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,E_OFF_RATING_RANK,E_DEF_RATING_RANK,E_NET_RATING_RANK,E_AST_RATIO_RANK,E_OREB_PCT_RANK,E_DREB_PCT_RANK,E_REB_PCT_RANK,E_TOV_PCT_RANK,E_USG_PCT_RANK,E_PACE_RANK
0,1086,Joe Stephens,2,1,1,0.500,4.9,84.6,102.0,-17.4,0.0,0.250,0.111,0.176,37.500,0.279,116.28,435,420,8,206,427,430,167,423,429,1,262,28,442,17,1
1,701,Frankie King,7,0,7,0.000,8.5,112.0,111.0,1.0,18.4,0.065,0.156,0.111,11.029,0.130,115.78,414,436,57,436,375,19,411,174,144,199,155,177,14,386,2
2,917,Jack Haley,20,6,14,0.300,3.7,116.8,111.9,5.0,9.5,0.169,0.284,0.222,3.820,0.224,110.71,371,374,119,343,438,4,416,80,312,6,7,6,1,92,3
3,725,Dell Demps,2,0,2,0.000,4.6,38.2,91.9,-53.7,0.0,0.000,0.000,0.000,0.000,0.185,110.70,435,436,18,436,430,443,9,444,429,434,438,441,1,213,4
4,968,Priest Lauderdale,35,26,9,0.743,5.1,102.7,106.4,-3.8,8.1,0.093,0.162,0.127,24.980,0.284,110.30,324,211,78,53,420,202,301,271,350,122,149,136,338,13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,770,Mark West,70,36,34,0.514,13.7,98.5,94.9,3.5,6.9,0.092,0.147,0.121,18.865,0.137,84.21,166,139,302,197,299,342,21,109,379,123,179,155,338,374,442
442,238,Tyrone Hill,74,40,34,0.541,34.9,103.8,98.5,5.3,9.2,0.131,0.245,0.188,14.676,0.178,84.13,137,110,302,177,69,163,67,72,325,25,25,19,14,235,443
443,1099,Matt Steigenga,2,1,1,0.500,5.9,89.1,108.9,-19.8,22.5,0.000,0.167,0.111,22.523,0.341,82.67,435,420,8,206,409,424,369,429,99,434,133,177,338,1,444
444,93,None,1,1,0,1.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.00,440,420,1,1,445,445,1,194,429,434,438,441,1,443,445


In [26]:
start_header = 1997
start_tail = 98
end = 2020
pem = pd.DataFrame()
while start_header < end:
    season_string = str(start_header) + '-' + str(start_tail)[-2:]
    print(season_string)
    temp = playerestimatedmetrics.PlayerEstimatedMetrics(season=season_string).get_data_frames()[0]
    temp['season'] = season_string
    if pem.empty:
        pem=temp
    else:
        pem = pem.append(temp, ignore_index=True)
    start_header += 1
    start_tail += 1

1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20


In [29]:
#Save
pem.to_pickle('./data/pem.pkl')

In [38]:
team_ids = teams_df['id']
start_header = 1997
start_tail = 98
end = 2020
team_roster = pd.DataFrame()
while start_header < end:
    season_string = str(start_header) + '-' + str(start_tail)[-2:]
    for tid in team_ids:
        #This is to avoid overloading api
        time.sleep(1)
        temp = commonteamroster.CommonTeamRoster(team_id=tid, season=season_string, timeout=600).get_data_frames()[0]
        temp['season'] = season_string
        if team_roster.empty:
            team_roster=temp
        else:
            team_roster = team_roster.append(temp, ignore_index=True)
    start_header += 1
    start_tail += 1


In [25]:
commonallplayers.CommonAllPlayers().get_data_frames()[0]

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,0,,,,,Y,00
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,0,,,,,Y,00
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,0,,,,,Y,00
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,0,,,,,Y,00
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,0,,,,,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4592,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,0,,,,,Y,01
4593,78647,"Zoet, Jim",Jim Zoet,0,1982,1982,HISTADD_jim_zoet,0,,,,,Y,00
4594,78648,"Zopf, Bill",Bill Zopf,0,1970,1970,HISTADD_zip_zopf,0,,,,,Y,00
4595,1627826,"Zubac, Ivica",Ivica Zubac,1,2016,2020,ivica_zubac,1610612746,LA,Clippers,LAC,clippers,Y,01


In [40]:


team_roster.to_pickle('../data/pulled/team_roster.pkl')